<h1><center> DISEÑO DE PILOTES PREEXCAVADOS EN SUELO </center></h1>

<div style="text-align: right"> Desarrolado por David León Vanegas - davidleon@ingetec.com.co </div> 

<div style="text-align: justify"> 
Hoja de calculo de pilotes preexcavados en suelos a partir de la metodología $\alpha$ y $\beta$ para el calculo de la resistencia por fricción y la formulación de O'Neill & Reese para el cálculo de la resistencia por punta. 
</div> 

In [1]:
# carga de paquetes necesarios para el calculo
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import CheckButtons
import pandas as pd
from numba import njit
from ipywidgets import *
from IPython.display import display
from notebook.auth import passwd

#%matplotlib nbagg
%matplotlib notebook
#%matplotlib inline

In [ ]:
passwd()

### 1. DEFINICIÓN DE PARAMETROS DEL SUELO

Numero de estratos en los que se dividide el suelo hasta la profundidad del pilote a analizar

In [ ]:
def f(Estratos=2):
    return Estratos
est = interactive(f, Estratos=[1, 2, 3, 4, 5])
display(est)

<div style="text-align: justify"> 
Definción de las propiedades geoténicas de cada uno de los materiales a lo largo del pilote, así como la profundidad en donde se ubica el nivel freático.
</div> 

In [ ]:
estratos = est.result # devuelve el resultado de la funcion el return
des, col, es, gam, su_min, su_max, c = [],[],[],[],[],[],[]
for i in range(estratos):
    des.append(widgets.Text(description='Tipo Material:'))
    col.append(widgets.ColorPicker(concise=False, description='Color',disabled=False))
    es.append(widgets.Text(description='Espesor $(m)$:'))
    gam.append(widgets.Text(description=r'$\gamma \, (kN/m^3)$:'))
    su_min.append(widgets.Text(description='Su $(kPa)$:', placeholder='valor minimo', layout=Layout(width='200px')))
    su_max.append(widgets.Text(description='—', placeholder='valor maximo', layout=Layout(width='200px')))
    c.append(widgets.VBox([des[i],col[i],es[i],gam[i], 
                           widgets.HBox([su_min[i],su_max[i]], layout=Layout(align_content='stretch') )]))
    HTML('<style> .widget-text { width: auto; } </style>')

accordion = widgets.Accordion(children=[c[i] for i in range(estratos)])
for i in range(estratos):
    accordion.set_title(i, 'Material '+str(i+1))
display(accordion)

freatico = widgets.Text(description='Nivel freatico: ')
display(freatico)

<div style="text-align: justify"> 
Perfil estratigrafico a lo largo de la profundidad de análisis.
</div> 

In [ ]:
# guardando informacion recibida de los estratos en array numpy
resumen = np.zeros((estratos, 6)).astype(object)
for i in range(estratos):
    if i == 0:
        resumen[i,1] = float(es[i].value)
    else:
        resumen[i,0] = resumen[i-1,1]
        resumen[i,1] = resumen[i,0] + float(es[i].value)
    resumen[i,2] = str(des[i].value)
    resumen[i,3] = float(gam[i].value)
    resumen[i,4] = float(su_min[i].value)
    resumen[i,5] = float(su_max[i].value)

#print(resumen)
NF = float(freatico.value)

fig, axes = plt.subplots(1, figsize=(5,5))
bbox_props = dict(boxstyle="round", fc="w", ec="0.5", alpha=0.9)
for i in range(estratos):
    axes.plot([0,10], [resumen[i,1], resumen[i,1]], '--r')
    axes.text(8.5,resumen[i,1],str(resumen[i,1])+' m')
    axes.fill([0, 10, 10, 0], [resumen[i,0], resumen[i,0], resumen[i,1], resumen[i,1]],color=col[i].value)
    axes.text(5, resumen[i,0]+float(es[i].value)/2, 
              resumen[i,2]+'\n'+r'$\gamma = $'+str(resumen[i,3])+r' $kN/m^3$'+'\n $S_u = $'+str(resumen[i,4])+'-'+str(resumen[i,5])+r' $kPa$',
              ha="center", va="center", size=10, bbox=bbox_props)

axes.plot([0,10],[NF,NF],'-b')
axes.text(0.5,NF,'NF='+str(NF)+'m')
axes.set(Title='PERFIL ESTRATIGRAFICO', ylabel="Profundidad (m)")
axes.get_xaxis().set_visible(False)
axes.set(xlim=(0, 10), ylim=(resumen[-1,1],0))
axes.plot()

### 2. Definicion de caracteristicas del pilote

<div style="text-align: justify"> 
Seleccionar el numero de diámetros de pilote diferentes a analizar
</div> 

In [ ]:
ndiam = widgets.Dropdown(options=['1','2','3','4','5','6','7','8'],
                       value='4', description='N° diametros')
display(ndiam)

<div style="text-align: justify"> 
Seleccionar los diámetros analizar, así como otras propiedades del pilote como la profundidad $z_{fric}$ en donde se desprecia la resistencia a la fricción y la profundiad crítica $z_{cri}$, dada en numero de diámetros para la cual la resisencia por fricción es suelos granulares alcanza su limite.
</div> 

In [ ]:
ndi = int(ndiam.value)
diam = []
for i in range(ndi):
    diam.append(widgets.Text(description='Diam '+str(i+1)+'(m):'))
boxdiam = widgets.VBox(diam)

z_fri = widgets.Text(description='Zfriccion (m):', value='1.5')
z_cri = widgets.Dropdown(options=['10','15','20'],
                       value='10', description='Zcritica (D):')
capa = widgets.Dropdown(options=['0.1','0.1','0.25','0.5','1.0'],
                       value='0.5', description='Capa $(m)$')

boxpilote = widgets.VBox([z_fri, z_cri, capa])
display(boxdiam, boxpilote)

### 3. Cálculo de la capacidad portante del pilote

<div style="text-align: justify"> 
La metodología de análisis empleada corresponde a la (XX), en donde la carga última de un pilote ($P_u$) está dada por la suma de la resistencia última por fuste ($P_{su}$) y por punta ($P_{bu}$), menos el peso del pilote ($W$).
</div> 

$$P_u = P_{su} + P_{bu} - W$$

<div style="text-align: justify"> 
La resistencia última por fuste ($P_{su}$) se evalua integrando la resistecia al corte en la interfaz suelo-pilote ($\tau_{a}$) a lo largo del área superficial del fuste, esta resistencia al corte puede evaluarse a través de la expresión de Coulomb.
</div> 

$$\tau_{a} = c_a + \sigma_{n}\tan{\phi_a}$$

<div style="text-align: justify"> 
En donde $c_a$ es la adhesión, $\sigma_n$ es el esfuerzo normal entre el pilote y el suelo y $\phi_a$ es el ángulo de fricción entre el pilote y el suelo. <br>
El esfuerzo normal $\sigma_n$ se relaciona a través del esfuerzo vertical ($\sigma_v$) a traves de un coeficiente de presión lateral ($K_s$)
</div>

$$\sigma_n = K_s \sigma_a$$

<div style="text-align: justify"> 
De esta manera la resistencia última por fueste puede determinarse a integrar a lo largo de la longitud del pilote la resistencia al corte entre el pilote y el suelo.
</div>

$$P_{su} = \int_{0}^{L}{\pi D (c_a + \sigma_v K_s \tan{\phi_a})dz} \qquad (1)$$

<div style="text-align: justify"> 
Para el cálculo de la resistencia por punta ($P_{bu}$) se suele usar la téoria de capacidad protante, por lo que esta magnitud se puede calcular a partir de la siguiente expresión.
</div>

$$P_{bu} = A_b (cN_c + \sigma_{vb}N_q + \frac12 \gamma d N_{\gamma}) \qquad (2)$$

<div style="text-align: justify"> 
En donde $A_b$ es el área de la base del pilote, $c$ es la cohesión, $\sigma_{vb}$ es el esfuerzo vertical en el suelo al nivel de la base del pilote, $\gamma$ es el peso unitario del suelo, $d$ es el diámetro del pilote y $N_c, N_q, N_{\gamma}$ son los factores de capacidad portante, los cuales son función del ángulo de fricción interno del suelo y la geometría del pilote.
<br> A partir de las ecuaciones (1) y (2) se obtiene la carga útima del pilote.
</div>

$$P_{u} = \int_{0}^{L}{\pi D (c_a + \sigma_v K_s \tan{\phi_a})dz} +
A_b (cN_c + \sigma_{vb}N_q + \frac12 \gamma d N_{\gamma}) - W  \qquad(3) $$

#### 3.1. Pilote en suelo fino - condición no drenada

<div style="text-align: justify"> 
Para pilotes en suelos finos, la capacidad de carga en condición no drenada es generalmente la condición critica (Para arcillas altamente sobreconsolidadas la condición crítica es la condición drenada). Si el suelo fino se encuentra saturado, el ángulo de fricción no drenado $\phi_u$ tiene un valor de cero y por lo tanto el ángulo $\phi_a$ se puede tomar como cero, para este valor del ángulo de fricción $N_q = 1$ y $N_{\gamma} = 0$, reduciendose entonces la Ecuación (3) a la siguiente expresión.
</div>

$$P_{u} = \int_{0}^{L}{\pi D c_a}dz + A_b (c_uN_c + \sigma_{vb})  - W $$

<div style="text-align: justify"> 
En donde $c_u$ es la resistencia al corte no drenada de suelo al nivel de la base del pilote y $c_a$ es la adhesión no drenada suelo-piote.
<br> La expresión anterior se puede simplificar para pilotes que no presentan ampliación de la sección en la base, debido a que $A_b\sigma_{vb} \approx W$, e igualmente se puede remplazar la integral a lo largo de la longitud del pilote por la sumatoria en capas de ancho $\Delta z$.
</div>

$$P_{u} = \sum{\pi D c_a} \Delta z + A_b c_uN_c  \qquad(4) $$

##### 3.1.1. Resistencia por punta

<div style="text-align: justify"> 
Para análisis en condición no drenada en suelos finos se puede seguir la metodología de O'Neill & Resse (1998) en esfuerzos totales, en donde para pilotes con una logitud mayor o igual a tres diaámetros, el factor de capacidad portante $N_c$ se pude calcular por medio de la siguiente ecuación. 
</div>

$$N_c = \begin{cases}
        6.5, & c_u < 47.8 kPa \\[1ex]
        8.0, & 47.8 kPa \leqslant c_u \leqslant 95.8 kPa \\[1ex]
        9.0, & c_u > 95.8 kPa
        \end{cases}$$

##### 3.1.2. Resistencia por fuste

<div style="text-align: justify"> 
La adhesión suelo-pilote $c_a$ puede calcular por medio de $c_a=\alpha c_u$, en donde $\alpha$ es un coeficiente que relaciona la adhesión con la resistencia al corte no drenada del suelo. El cálculo de la resistencia por fuste usando esta expresión se conoce como el "_método $\alpha$_".
<br> El coeficiente $\alpha$ se determina normalmente en pruebas de carga, sin embargo existen un gran número de aproximaciones empíricas.
</div>

###### Calculo coeficiente $\alpha$ - FCHW

$$\alpha = \begin{cases}
        0.0, & z < 5 ft \approx 1.5 m \\[1ex]
        0.55, & \frac{c_u}{P_a} \leqslant 1.5  \\[1ex]
        0.55 - 0.1 \left(\frac{c_u}{P_a} - 1.5 \right), & 1.5 < \frac{c_u}{P_a} \leqslant 2.5
        \end{cases}$$

###### Calculo coeficiente $\alpha$ - Metro Bogotá

$$\alpha = \begin{cases}
        0.0, & z < 1.5 m \\[1ex]
        0.2 + 0.8\exp\left({0.35 -2\frac{c_u}{P_a}}\right), & \frac{c_u}{P_a} > 0.175  \\[1ex]
        1.0, &  \frac{c_u}{P_a} \leqslant 0.175
        \end{cases}$$

Función para el calculo de capacidad portante

In [ ]:
# organizando informacion de diametros y el pilote
D = [float(diam[i].value) for i in range(ndi)]
Zf = float(z_fri.value)
Zc = int(z_cri.value)
dcapa = float(capa.value)

ndatos = int(resumen[-1,1]/dcapa)
tabla = np.zeros((ndatos,9))
tabla[:,0] = np.linspace(0, resumen[-1,1]-dcapa, ndatos) #zi
tabla[:,1] = np.linspace(dcapa, resumen[-1,1], ndatos) #zf
tabla[:,2] = np.mean([tabla[:,0], tabla[:,1]], axis=0) #zm
for i in range(estratos):
    idz = np.where((tabla[:,1] > resumen[i,0]) & (tabla[:,1]<= resumen[i,1]))[0]
    tabla[idz,3] = resumen[i,3] # peso unitario
    tabla[idz,4] = resumen[i,4] # Su min
    tabla[idz,6] = resumen[i,5] # Su max

tabla[:,5] = np.mean([tabla[:,4], tabla[:,6]], axis=0) # Su promedio

idnf = np.where(tabla[:,0]>= NF)[0]
tabla[idnf,3] -= 9.81 # peso unitario efectivo

# funcion para el calculo de capacidad ultima del pilote
def pile_pu(tabla, Zc, Zf, D, alp):
    """Funcion que calcula la capacidad portante ultima a lo largo del pilote para los
        diametros dados - SOLO SUELO FINO EN CONDICION NO DREANADA!!!
        calcula para el valor minimo maximo y medio de Su
        alp define la manera como se calcula el coeficiente alpha
            alp='Hoja MetroBog'  => Hoja de calculo metro
            alp='FHWA'  => FHWA"""
    
    # Calculo de variables preliminares
    dz = tabla[0,1] - tabla[0,0]
    dsigma = tabla[:,3]*dz
    
    ndi = len(D) # numero de diametros
    ndatos = len(tabla[:,0]) # numero de datos - capas
    Psu = [0,0,0]
    Pbu = [0,0,0]
    
    for n in range(3):
        # ciclo para moverse por el valor de Su
        
        # Iniciar array para carga ultima por punta y por friccion
        Psu[n] = np.zeros((ndatos,ndi))
        Pbu[n] = np.zeros((ndatos,ndi))

        for j in range(ndi):
            # calculo del esfuerzo vertical
            for i in range(ndatos):
                if i == 0:
                    tabla[i,7] = dsigma[i] # esfuerzo en la base
                    tabla[i,8] = dsigma[i]/2 # esfuerzo en la mitad de la capa
                else:
                    tabla[i,7] = dsigma[i] + tabla[i-1,7] # esfuerzo en la base
                    tabla[i,8] = dsigma[i]/2 + tabla[i-1,7] # esfuerzo en la mitad de la capa

            # restringuiendo el esfuerzo limite para suelos granualares
            idzc = np.where(tabla[:,2] >= Zc*D[j])[0]
            sigma_lim = tabla[idzc[0] - 1,6] # esfuerzo limite
            tabla[idzc,8] = sigma_lim

            # calculo del coeficiente alpha
            if alp == 'Hoja MetroBog':
                # calculo segun hoja de pilotes
                alpha = 0.2 + 0.8*np.exp(0.35 - 2*tabla[:,n+4]/101.3)
                idalphalim = np.where(alpha >= 1)[0]
                alpha[idalphalim] = 1.0
            elif alp == 'FHWA':
                # calculo segun FHWA
                alpha = 0.55 - 0.1*(tabla[:,n+4]/101.3 - 1.5)
                idalphalim = np.where(tabla[:,n+4]/101.3 <= 1.5)[0]
                alpha[idalphalim] = 0.55
            
            idZf= np.where(tabla[:,2] < Zf)[0]
            alpha[idZf] = 0.0
            

            # resistencia nominales - sin importar el diametro del pilote
            qs = alpha * tabla[:,n+4]*dz
            qb = 9 * tabla[:,4]
            qb[idZf] = 0.0
            Qs = np.zeros(ndatos)
            for i in range(ndatos):
                if i == 0:
                    Qs[i] = qs[i]
                else:
                    Qs[i] = Qs[i-1] + qs[i]

            # resistencia para diametro dado
            # para el diametro D[0]
            Psu[n][:,j] = Qs * np.pi * D[j]
            Pbu[n][:,j] = qb * np.pi * D[j] ** 2 / 4
    
    return Psu, Pbu

Seleccionar el método de calculo del coeficiente $\alpha$.

In [ ]:
alp_metodo = widgets.Dropdown(options=['FHWA','Hoja MetroBog'],
                       value='FHWA', description='Calculo '+ r'$\alpha$')
display(alp_metodo)

In [ ]:
alp = alp_metodo.value
# calculo de capacidad ultima
Psu, Pbu = pile_pu(tabla, Zc, Zf, D, alp)
# Organizando carga ultima
Pui = Psu[0] +Pbu[0]
Pum = Psu[1] + Pbu[1]
Pus = Psu[2] + Pbu[2]
# carga admisibles
Padi = Psu[0]/3 + Pbu[0]/3
Padm = Psu[1]/3 + Pbu[1]/3
Pads = Psu[2]/3 + Pbu[2]/3

# Abaco de capacidad de carga de cada pilote
color = ['blue','green','red','yellow','cyan','lime','magenta',
         'orange','chocolate','olive']
fig, axes = plt.subplots(1, figsize=(7,6))
di = np.zeros(len(D)).astype(object)
dm = np.zeros(len(D)).astype(object)
ds = np.zeros(len(D)).astype(object)
lacheck2 = ('minimo','medio','maximo')
boocheck2 = (False, True, False)
for i in range(ndi):
    di[i], = axes.plot(Padi[:,i], tabla[:,0], '--', color=color[i], visible=False)
    dm[i], = axes.plot(Padm[:,i], tabla[:,0], color=color[i], label = 'D = '+str(D[i])+' m')
    ds[i], = axes.plot(Pads[:,i], tabla[:,0], '--', color=color[i], visible=False)

plt.subplots_adjust(left=0.25)    
axes.set(Title='Abaco cimentacion profunda - Caso estatico', 
         xlabel='Capacidad de carga pilote individual $(kN)$', 
         ylabel='Longitud del pilote ($m$) ')
axes.set(xlim=(0, np.max(Pads) + 100), ylim=(resumen[-1,1],0))
axes.grid(True, linestyle=':')
axes.legend()

# checkbutton 2
rax2 = plt.axes([0.05, 0.7, 0.11, 0.15], fc='lightgoldenrodyellow', title='Su')
check2 = CheckButtons(rax2, lacheck2, boocheck2)

def su(label2):
    for i in range(ndi):
        if label2 == lacheck2[0]:
            di[i].set_visible(not di[i].get_visible())
        if label2 == lacheck2[1]:
                dm[i].set_visible(not dm[i].get_visible())
        if label2 == lacheck2[2]:
                ds[i].set_visible(not ds[i].get_visible())      
        plt.draw()

check2.on_clicked(su)

plt.show()

In [ ]:
# correr para ocultar todo el codigo del programa
%%html
<style>
div.input {
    display:none;
}
</style>